<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/Spacial/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install R and IRkernel
!apt-get install r-base
!R -e "install.packages('IRkernel', repos='http://cran.us.r-project.org')"
!R -e "IRkernel::installspec(user = FALSE)"

Colab menu: Runtime > Change runtime type.
Runtime type: R.

In [ ]:
rm(list=ls())

# load the libraries with all dependencies
if(!require('giscoR')) install.packages('giscoR', dependencies=T); library('giscoR')

In [ ]:
library(giscoR)
library(sf)
library(dplyr)

# Different resolutions
GRC_res60 <- gisco_get_countries(resolution = "60", country = "GRC") %>%
  mutate(res = "60M")
GRC_res20 <-
  gisco_get_countries(resolution = "20", country = "GRC") %>%
  mutate(res = "20M")
GRC_res10 <-
  gisco_get_countries(resolution = "10", country = "GRC") %>%
  mutate(res = "10M")
GRC_res03 <-
  gisco_get_countries(resolution = "03", country = "GRC") %>%
  mutate(res = "03M")


GRC_all <- bind_rows(GRC_res60, GRC_res20, GRC_res10, GRC_res03)

# Plot ggplot2

library(ggplot2)

ggplot(GRC_all) +
  geom_sf(fill = "tomato") +
  facet_wrap(vars(res)) +
  theme_minimal()

In [ ]:


# Labels and Lines available

labs <- gisco_get_countries(
  spatialtype = "LB",
  region = "Africa",
  epsg = "3857"
)

coast <- gisco_get_countries(
  spatialtype = "COASTL",
  epsg = "3857"
)

# For zooming
afr_bbox <- st_bbox(labs)

ggplot(coast) +
  geom_sf(col = "deepskyblue4", linewidth = 3) +
  geom_sf(data = labs, fill = "springgreen4", col = "darkgoldenrod1", size = 5, shape = 21) +
  coord_sf(
    xlim = afr_bbox[c("xmin", "xmax")],
    ylim = afr_bbox[c("ymin", "ymax")]
  )

In [ ]:
# Get shapes
nuts3 <- gisco_get_nuts(
  year = "2021",
  epsg = "3035",
  resolution = "10",
  nuts_level = "3"
)

# Group by NUTS by country and convert to lines
country_lines <- nuts3 %>%
  group_by(
    CNTR_CODE
  ) %>%
  summarise(n = n()) %>%
  st_cast("MULTILINESTRING")

In [ ]:
# Use eurostat
library(eurostat)
popdens <- get_eurostat("demo_r_d3dens") %>%
  filter(TIME_PERIOD == "2021-01-01")

In [ ]:
# Merge data
nuts3_sf <- nuts3 %>%
  left_join(popdens, by = "geo")

nuts3_sf <- nuts3 %>%
  left_join(popdens, by = c("NUTS_ID" = "geo"))


# Breaks and labels

br <- c(0, 25, 50, 100, 200, 500, 1000, 2500, 5000, 10000, 30000)
labs <- prettyNum(br[-1], big.mark = ",")

# Label function to be used in the plot, mainly for NAs
labeller_plot <- function(x) {
  ifelse(is.na(x), "No Data", x)
}
nuts3_sf <- nuts3_sf %>%
  # Cut with labels
  mutate(values_cut = cut(values, br, labels = labs))


# Palette
pal <- hcl.colors(length(labs), "Lajolla")


# Plot
ggplot(nuts3_sf) +
  geom_sf(aes(fill = values_cut), linewidth = 0, color = NA, alpha = 0.9) +
  geom_sf(data = country_lines, col = "black", linewidth = 0.1) +
  # Center in Europe: EPSG 3035
  coord_sf(
    xlim = c(2377294, 7453440),
    ylim = c(1313597, 5628510)
  ) +
  # Legends
  scale_fill_manual(
    values = pal,
    # Label for NA
    labels = labeller_plot,
    drop = FALSE, guide = guide_legend(direction = "horizontal", nrow = 1)
  ) +
  # Theming
  theme_void() +
  # Theme
  theme(
    plot.title = element_text(
      color = rev(pal)[2], size = rel(1.5),
      hjust = 0.5, vjust = -6
    ),
    plot.subtitle = element_text(
      color = rev(pal)[2], size = rel(1.25),
      hjust = 0.5, vjust = -10, face = "bold"
    ),
    plot.caption = element_text(color = "grey60", hjust = 0.5, vjust = 0),
    legend.text = element_text(color = "grey20", hjust = .5),
    legend.title = element_text(color = "grey20", hjust = .5),
    legend.position = "bottom",
    legend.title.position = "top",
    legend.text.position = "bottom",
    legend.key.height = unit(.5, "line"),
    legend.key.width = unit(2.5, "line")
  ) +
  # Annotate and labs
  labs(
    title = "Population density in 2021",
    subtitle = "NUTS-3 level",
    fill = "people per sq. kilometer",
    caption = paste0(
      "Source: Eurostat, ", gisco_attributions(),
      "\nBased on Milos Popovic: ",
      "https://milospopovic.net/how-to-make-choropleth-map-in-r/"
    )
  )